In [158]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

In [159]:
# Read recipe inputs
STC_indicator = dataiku.Dataset("STC_indicator")
df_stc = STC_indicator.get_dataframe()

In [160]:
df_stc.tail()

,Date,Close,STC_tailored,STC_std
9187,2023-01-13 05:00:00+00:00,11541.48,95.16,98.07
9188,2023-01-17 05:00:00+00:00,11557.19,97.58,99.03
9189,2023-01-18 05:00:00+00:00,11410.29,98.79,99.52
9190,2023-01-19 05:00:00+00:00,11295.67,99.39,99.76
9191,2023-01-20 05:00:00+00:00,11619.03,99.70,99.88


# Position dans les deciles #

In [161]:
def get_deciles_list(col):
    '''Determiner les seuils des deciles.
    Pour rappel : 10 quantiles = décile '''

    decile_list = []

    for i in range(1,10):
        decile_list.append(col.quantile(i/10, interpolation="nearest"))

    return decile_list

In [162]:
l_deciles = get_deciles_list(df_stc["STC_tailored"])
print(l_deciles)

[0.24, 3.5, 13.75, 31.76, 53.87, 73.4, 88.91, 98.3, 99.95]


In [163]:
l_deciles = [round(e,1) for e in l_deciles]
print(l_deciles)

[0.2, 3.5, 13.8, 31.8, 53.9, 73.4, 88.9, 98.3, 100.0]


In [164]:
# La fonction qcut paramétrée à 10 retourne les intervalles des quantiles et situe la valeur dans celui-ci
# .right nous indique la partie droite de l'intervalle, qui nous servira à identifier en valeurs numériques
# la position de la valeur dans les décoles
raw_quantile_position = pd.qcut(df_stc["STC_tailored"], 10).apply(lambda x: round(x.right,1))
raw_quantile_position

0         0.2
1         0.2
2         0.2
3         0.2
4         0.2
        ...  
9187     98.3
9188     98.3
9189    100.0
9190    100.0
9191    100.0
Name: STC_tailored, Length: 9192, dtype: float64

In [165]:
def attribute_quantile_position(raw_l, quantiles_l):

    refined_quantile_list = []

    for e in raw_l:
        # The index() method returns the index of the specified element in the list.
        p = quantiles_l.index(e)

        # p+1 car sinon 1er décile à position 0
        # Avec .right() il n'y a que 9 intervalles pour 10 déciles
        refined_quantile_list.append(p+1)

    return refined_quantile_list

In [166]:
df_stc["STC_tailored_decile_position"] = attribute_quantile_position(raw_quantile_position,l_deciles)

Remarques :
<li>Le moindre défaut d'arrondi peut compromettre la bonne execution de cette fonction.</li>
<li>La même méthode sur STC standard donne exactement le même résultat. Pas besoin de répliquer</li>

# Lags & %var #

In [168]:
df_stc.head()

,Date,Close,STC_tailored,STC_std,STC_tailored_decile_position
0,1986-08-01 04:00:00+00:00,144.19,0.0,0.0,1
1,1986-08-04 04:00:00+00:00,142.52,0.0,0.0,1
2,1986-08-05 04:00:00+00:00,141.86,0.0,0.0,1
3,1986-08-06 04:00:00+00:00,140.54,0.0,0.0,1
4,1986-08-07 04:00:00+00:00,140.70,0.0,0.0,1


In [169]:
def generate_diff(col, shift) :

    shifted_col = col.shift(shift)

    l_res = []

    for e in range (0, len(col)):
        l_res.append(round(col[e]-shifted_col[e],2))

    return l_res

In [170]:
for i in [1,2,3,5]:
    colname1 = "STC_tailored_diff" + str(i)
    colname2 = "STC_std_diff" + str(i)
    colname3 = "STC_tailored_decile_position_Lag" + str(i)

    df_stc[colname1] = generate_diff(df_stc["STC_tailored"],i)
    df_stc[colname2] = generate_diff(df_stc["STC_std"],i)
    df_stc[colname3] = df_stc["STC_tailored_decile_position"].shift(i)

In [171]:
df_stc.tail()

,Date,Close,STC_tailored,STC_std,STC_tailored_decile_position,STC_tailored_diff1,STC_std_diff1,STC_tailored_decile_position_Lag1,STC_tailored_diff2,STC_std_diff2,STC_tailored_decile_position_Lag2,STC_tailored_diff3,STC_std_diff3,STC_tailored_decile_position_Lag3,STC_tailored_diff5,STC_std_diff5,STC_tailored_decile_position_Lag5
9187,2023-01-13 05:00:00+00:00,11541.48,95.16,98.07,8,4.85,1.94,8.0,9.47,5.80,7.0,12.87,13.53,7.0,16.70,59.93,7.0
9188,2023-01-17 05:00:00+00:00,11557.19,97.58,99.03,8,2.42,0.96,8.0,7.27,2.90,8.0,11.89,6.76,7.0,17.72,29.96,7.0
9189,2023-01-18 05:00:00+00:00,11410.29,98.79,99.52,9,1.21,0.49,8.0,3.63,1.45,8.0,8.48,3.39,8.0,16.50,14.98,7.0
9190,2023-01-19 05:00:00+00:00,11295.67,99.39,99.76,9,0.60,0.24,9.0,1.81,0.73,8.0,4.23,1.69,8.0,13.70,7.49,7.0
9191,2023-01-20 05:00:00+00:00,11619.03,99.70,99.88,9,0.31,0.12,9.0,0.91,0.36,9.0,2.12,0.85,8.0,9.39,3.75,8.0


In [172]:
df_stc.describe()

,Close,STC_tailored,STC_std,STC_tailored_decile_position,STC_tailored_diff1,STC_std_diff1,STC_tailored_decile_position_Lag1,STC_tailored_diff2,STC_std_diff2,STC_tailored_decile_position_Lag2,STC_tailored_diff3,STC_std_diff3,STC_tailored_decile_position_Lag3,STC_tailored_diff5,STC_std_diff5,STC_tailored_decile_position_Lag5
count,9192.000000,9192.000000,9192.000000,9192.000000,9191.000000,9191.000000,9191.000000,9190.000000,9190.000000,9190.000000,9189.000000,9189.000000,9189.000000,9187.000000,9187.000000,9187.000000
mean,2926.517291,51.366870,52.690381,5.399478,0.010848,0.010867,5.399086,0.021664,0.021724,5.398694,0.032417,0.032556,5.398302,0.053404,0.054018,5.397736
std,3490.883219,39.544583,43.098361,2.728475,3.547190,13.378702,2.728365,6.974636,23.486096,2.728255,10.233833,31.605580,2.728145,16.184094,43.546963,2.728172
min,128.430000,0.000000,0.000000,1.000000,-31.060000,-50.000000,1.000000,-55.890000,-75.000000,1.000000,-71.420000,-87.500000,1.000000,-88.480000,-96.880000,1.000000
25%,569.937500,7.655000,2.675000,3.000000,-1.020000,-1.120000,3.000000,-2.010000,-2.930000,3.000000,-3.120000,-5.470000,3.000000,-5.270000,-14.150000,3.000000
50%,1672.380000,53.865000,56.815000,5.500000,0.000000,0.000000,5.000000,0.000000,0.000000,5.000000,0.000000,0.000000,5.000000,0.000000,0.000000,5.000000
75%,3708.470000,94.770000,99.260000,8.000000,1.230000,1.240000,8.000000,2.430000,2.830000,8.000000,3.670000,5.470000,8.000000,6.110000,12.990000,8.000000
max,16573.340000,100.000000,100.000000,9.000000,22.840000,50.000000,9.000000,43.070000,75.000000,9.000000,58.730000,87.500000,9.000000,78.160000,96.880000,9.000000


# Nettoyage & écriture

In [173]:
df_stc.dropna(inplace=True)
df_stc.reset_index(drop=True, inplace=True)

In [0]:
# Write recipe outputs
stc_enriched = dataiku.Dataset("stc_enriched")
stc_enriched.write_with_schema(df_stc)